In [1]:
import pandas as pd
import xarray as xr

from scipy import optimize as opt
from scipy import stats

In [186]:
path = "../results/prec_miro_hist_hist_tretorno.nc"
path_1 = "../results/prec_miro_hist_hist_tretorno_1.nc"
df = xr.open_dataset(path).to_dataframe()
df_2 = df.copy().drop( ["PROBABILIDAD", "AÑO"], axis = 1 
    ).reset_index( "TIEMPO_RETORNO" )
#df_2 = df_2.reset_index( "DURACION" )
df_3 = df_2.copy().drop( ["INTENSIDAD", "TIEMPO_RETORNO"], axis = 1).groupby(
    ["south_north", "west_east", "DURACION"] ).mean()
cols = ["GEV_C", "GEV_LOC", "GEV_SCALE", "KTEST_P"]
df_3[ cols ] = None
df_3

LONGITUD    LATITUD GEV_C GEV_LOC GEV_SCALE  \
south_north west_east DURACION                                                  
0           0         1        -100.453476  18.135178  None    None      None   
                      2        -100.453476  18.135178  None    None      None   
                      3        -100.453476  18.135178  None    None      None   
                      4        -100.453476  18.135178  None    None      None   
                      5        -100.453476  18.135178  None    None      None   
...                                    ...        ...   ...     ...       ...   
111         131       20        -97.333252  20.628967  None    None      None   
                      21        -97.333252  20.628967  None    None      None   
                      22        -97.333252  20.628967  None    None      None   
                      23        -97.333252  20.628967  None    None      None   
                      24        -97.333252  20.628967  None    None      None   

                               KTEST_P  
south_north west_east DURACION          
0           0         1           None  
                      2           None  
                      3           None  
                      4           None  
                      5           None  
...                                ...  
111         131       20          None  
                      21          None  
                      22          None  
                      23          None  
                      24          None  

[354816 rows x 6 columns]

In [187]:
#d = [ 5/60, 10/60, 15/60, 20/60, 30/60, 40/60, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5,
#    6, 8, 10, 12, 14, 16, 18, 20, 22, 24 ] * 132 * 112 * 31
t = ( [5, 10, 25, 50, 100, 200, 500, 1000] + [None] * 23 ) * 112 * 132 * 24
#df_2["DURACION"] = d
df_2["TIEMPO_RETORNO"] = t
df_2 = df_2.dropna()
#df_2 = df_2.set_index("DURACION", append = True).sort_values(
#    ["south_north", "west_east", "DURACION", "TIEMPO_RETORNO"] )
df_2

TIEMPO_RETORNO    LONGITUD    LATITUD  \
south_north west_east DURACION                                          
0           0         1                    5.0 -100.453476  18.135178   
                      1                   10.0 -100.453476  18.135178   
                      1                   25.0 -100.453476  18.135178   
                      1                   50.0 -100.453476  18.135178   
                      1                  100.0 -100.453476  18.135178   
...                                        ...         ...        ...   
111         131       24                  50.0  -97.333252  20.628967   
                      24                 100.0  -97.333252  20.628967   
                      24                 200.0  -97.333252  20.628967   
                      24                 500.0  -97.333252  20.628967   
                      24                1000.0  -97.333252  20.628967   

                                INTENSIDAD  
south_north west_east DURACION              
0           0         1                0.0  
                      1                0.0  
                      1                0.0  
                      1                0.0  
                      1                0.0  
...                                    ...  
111         131       24               0.0  
                      24               0.0  
                      24               0.0  
                      24               0.0  
                      24               0.0  

[2838528 rows x 4 columns]

In [188]:
i = 90
j = 90
for k in df_3.index.get_level_values("DURACION").unique():
    params = stats.genextreme.fit(df.loc[ (i, j, k), "INTENSIDAD"] )
    pvalue = stats.kstest(df.loc[ (i, j, k), "INTENSIDAD"],
        stats.genextreme(*params).cdf ).pvalue
    df_3.loc[ (i, j, k), cols ] = [*params] + [pvalue]
    df_2.loc[ (i, j, k), "INTENSIDAD" ] = stats.genextreme(
        *df_3.loc[ (i, j, k), cols[:-1] ] ).isf(
        1 / df_2.loc[ (i, j, k), "TIEMPO_RETORNO" ].values )

In [191]:
df_2.loc[(90, 90, 3)]

TIEMPO_RETORNO   LONGITUD    LATITUD  \
south_north west_east DURACION                                         
90          90        3                    5.0 -98.309807  20.160126   
                      3                   10.0 -98.309807  20.160126   
                      3                   25.0 -98.309807  20.160126   
                      3                   50.0 -98.309807  20.160126   
                      3                  100.0 -98.309807  20.160126   
                      3                  200.0 -98.309807  20.160126   
                      3                  500.0 -98.309807  20.160126   
                      3                 1000.0 -98.309807  20.160126   

                                INTENSIDAD  
south_north west_east DURACION              
90          90        3          14.722277  
                      3          16.814188  
                      3          19.710766  
                      3          22.058870  
                      3          24.572220  
                      3          27.271442  
                      3          31.156225  
                      3          34.357651